In [1]:
import sqlite3
import random
import sqlite3
import PySimpleGUI as sg
from datetime import datetime
import sys

con = sqlite3.connect('Project.db')
cur = con.cursor()

In [2]:
def window_login():
    
    layout = [[sg.Text('Welcome to the Intergalactic Spacecraft Application. Please enter your credentials.')],
              [sg.Text('ISN:',size=(10,1)), sg.Input(size=(10,1), key='isn')],
              [sg.Text('Password:',size=(10,1)), sg.Input(size=(10,1), key='password',password_char='*')],
              [sg.Button('Login')]]

    return sg.Window('Login', layout)

def window_manager():
    
    layout = [[sg.Text(userName + " "+ userSurname + " " + userIsn)],
              [sg.Button('Purchase Operations')],
              [sg.Button('Project Directors')],
              [sg.Button('Spacecraft Projects')],
              [sg.Button('Workers')],
              [sg.Button('Manufactured Spacecrafts')],
              [sg.Button('Logout')]]

    return sg.Window('Manager Window', layout)

def window_worker():
    
    layout = [[sg.Text(userName+" "+userSurname+" "+userIsn)],
              [sg.Button('View Ratings')],
              [sg.Button('Logout')]]
    
    return sg.Window('Worker Window',layout)

def window_customer():
    
    layout = [[sg.Text(userName + " "+ userSurname + " " + userIsn)],
              [sg.Button('Spacecrafts')],
              [sg.Button('Logout')]]

    return sg.Window('Customer Window', layout)

def window_salesRep():
    layout = [[sg.Text(userName + " "+ userSurname + " " + userIsn)],
              [sg.Button('Purchase Operations')],
              [sg.Button('Pending Purchase Operations')],
              [sg.Button('Logout')]]

    return sg.Window('Sales Representative Window', layout)
def window_projectDirector():
    layout = [[sg.Text(userName + " "+ userSurname + " " + userIsn)],
              [sg.Button('List My Projects')],
              [sg.Button('Create a Project')],
              [sg.Button('Assign Workers to Teams')],
              [sg.Button('Logout')]]
              

    return sg.Window('Project Director Window', layout)


In [3]:
def button_login(values):
    
    global userIsn
    global userName
    global userSurname
    global userType
    global window

    
    uisn = values['isn']
    userpass = values['password']
    if uisn == '':
        sg.popup('ISN cannot be empty')
    elif userpass == '':
        sg.popup('Password cannot be empty')
        window = window_login()
    else:
        # Checking if this is a valid user
        cur.execute('SELECT ISN, Name, Surname FROM USER WHERE ISN = ? AND password = ?', (uisn,userpass,))
        row = cur.fetchone()
        if row is None:
            sg.popup('ID or password is wrong!')
            window = window_login()
        
        else:
            userIsn = row[0]
            
           
            userName = row[1]
            
            userSurname = row[2]
            
            cur.execute('SELECT GMISN FROM GENERAL_MANAGER WHERE GMISN = ?', (uisn,))
            row_manager = cur.fetchone()
            if row_manager == None: #That means he/she is not a manager. We have to search for other types.
                cur.execute('SELECT WISN FROM WORKER WHERE WISN = ?', (uisn,))
                row_worker = cur.fetchone()
                if row_worker == None: #That means he/she is not a worker too!
                    cur.execute('SELECT CISN FROM CUSTOMER WHERE CISN = ?', (uisn,))
                    row_customer = cur.fetchone()
                    if row_customer == None: #That means he/she is not a customer. We have to search for other types.
                        cur.execute('SELECT SRISN FROM SALES_REPRESENTATIVE WHERE SRISN = ?', (uisn,))
                        row_sales = cur.fetchone()
                        if row_sales == None: #That means he/she is not a sales representative too!
                            cur.execute('SELECT PDISN FROM PROJECT_DIRECTOR WHERE PDISN =?', (uisn,))
                            row_pds = cur.fetchone()
                            if row_pds == None:
                                sg.popup('User type error! Please contact the admin.') #Raising an error
                            else:
                                userType = 'Project Director'
                                sg.popup('Welcome, ' + userName + ' (Project Director)')
                                window.close()
                                window = window_projectDirector()
                        else:
                            # this is a sales representative
                            userType = 'Sales Representative'
                            sg.popup('Welcome, ' + userName + ' (Sales Representative)')
                            window.close()
                            window = window_salesRep()
                    else:
                        # this is a Customer
                        userType = 'Customer'
                        sg.popup('Welcome, ' + userName + ' (Customer)')
                        window.close()
                        window = window_customer()
                else:
                    # this is a worker
                    userType = 'Worker'
                    sg.popup('Welcome, ' + userName + ' (Worker)')
                    window.close()
                    window = window_worker()
            else:
                # this is a Manager
                userType = 'Manager'
                sg.popup('Welcome, ' + userName + ' (Manager)')
                window.close()
                window = window_manager()
        
            
            
                
def button_list_purchases(values = None):
    global window
    operations = []
    operations.append(('Invoice', 'Plate', 'SR_ISN', 'Tr_date'))
    for row in cur.execute('SELECT Invoice, Plate, SR_ISN, Tr_date FROM PURCHASE'):
        operations.append(row)
        
    layout = [[sg.Text('Purchase Opreations')],
              [sg.Listbox(operations,size=(40,7),key='operations')],
              [sg.Button('Return To Main')]]
    
    window = sg.Window('Purchase Operations',layout)
    

def button_project_directors(values = None):
    global window
    directors = []
    directors.append(('PDISN', 'Salary', 'S_no', 'Work_hours','Experience'))
    for row in cur.execute('SELECT PDISN, Salary, S_no, Work_hours, Experience FROM PROJECT_DIRECTOR'):
        directors.append(row)
        
    layout = [[sg.Text('Project Directors')],
              [sg.Listbox(directors,size=(40,7),key='directors')],
              [sg.Button('Return To Main'), sg.Button('Insert Project Director')]]
    
    window = sg.Window('Project Directors',layout)

def button_spacecraft_projects(values = None):
    global window
    projects = []
    projects.append(('Pr_No', 'PD_ISN', 'Start', 'End','Team_No'))
    for row in cur.execute('SELECT Pr_No, PD_ISN, Start, End, Team_No FROM PERFORMED_PROJECT'):
        projects.append(row)
        
    layout = [[sg.Text('Spacecraft Projects')],
              [sg.Listbox(projects,size=(40,7),key='projects')],
              [sg.Button('Return To Main'), sg.Button('Select')]]
    
    window = sg.Window('Spacecraft Projects',layout)
    
    
def button_workers(values = None):
    global window

    
    workers = []
    workers.append(('WISN', 'Shift_time', 'Team_no', 'Salary'))
    for row in cur.execute('SELECT WISN, Shift_time, Team_no, Salary FROM WORKER'):
        workers.append(row)
    
    layout = [[sg.Text('Workers')],
              [sg.Listbox(workers,size=(40,7),key='workers')],
              [sg.Button('Return To Main'),sg.Button('Delete Worker'),sg.Button('Update Worker'),sg.Button('Insert New Worker')]]
    
    window = sg.Window('Workers',layout)
    

def button_spacecrafts(values = None):
    global window
    spacecrafts = []
    spacecrafts.append(('Lic_Plate', 'Num_Seats', 'Brand', 'Price', 'Cost','Pr_No'))
    for row in cur.execute('SELECT Lic_Plate, Num_Seats, Brand, Price, Cost,Pr_No FROM SPACE_CRAFT_CREATION'):
        spacecrafts.append(row)
        
    layout = [[sg.Text('Manufactured Spacecrafts')],
              [sg.Listbox(spacecrafts,size=(40,7),key='spacecrafts')],
              [sg.Button('Return To Main')]]
    
    window = sg.Window('Manufactured Spacecrafts',layout)
    

def button_insert_worker(values):
    global window 
    
    
    Name = values['Name']
    Surname = values['Surname']
    shift_time = values['shift_time']
    team_no = values['team_no']
    salary= values['salary']
    ISN = ""
    for i in range(8):
        ISN += str(random.randint(0,9))
    password = ""
    for i in range(5):
        password+=str(random.randint(0,9))
    if Name == '':
        sg.popup('Name cannot be empty!')
    elif Surname == '':
        sg.popup('Surname cannot be empty!')
    elif salary == '':
        sg.popup('Please write the salary amount. Write 0 for no salary.')
    elif not salary.isnumeric():
        sg.popup('Salary should be numeric.')
    else:
        
       
        try:
            cur.execute('INSERT INTO USER VALUES (?,?,?,?)',(ISN,Name,Surname,password,))
            con.commit()
            cur.execute('INSERT INTO WORKER VALUES (?,?,?,?)',(ISN,shift_time,team_no,salary,))
            con.commit()
        
            sg.popup('Successfully inserted ' + Name + ' ' + Surname + ' with id: ' + str(ISN) + ', password is ' + password)
        except: 
            sg.popup('ISN must be unique, try again')
        
    window_insert_worker(values)
    
def window_insert_worker(values):
    global window
    
    layout = [[sg.Text('Name:',size=(10,1)) , sg.Input(key='Name',size=(10,1))],
              [sg.Text('Surname:',size=(10,1)) , sg.Input(key='Surname',size=(10,1))],
              [sg.Text('Shift Time',size=(10,1)) , sg.Input(key='shift_time',size=(10,1))],
              [sg.Text('Team No',size=(10,1)) , sg.Input(key='team_no',size=(10,1))],
              [sg.Text('Salary',size=(10,1)) , sg.Input(key='salary',size=(10,1))],
              [sg.Button('Insert Worker') , sg.Button('Return To Main')]]
    
    
    window = sg.Window('Insert New Worker', layout)
    
def button_delete_worker(values):
    global window
    
   
    if values['workers'] == [] or values['workers']== [[]] or values['workers'] == "":
        sg.popup('Please Select a Worker')
        button_workers(values = None)
        return 
    
    ISN = values['workers'][0][0]
    
    cur.execute('DELETE FROM WORKER WHERE WISN=?',(ISN,))
    con.commit()
    cur.execute('DELETE FROM USER WHERE ISN=?',(ISN,))
    con.commit()

    sg.popup('Succesfuly Deleted')
    layout = [[sg.Text('Worker is Deleted')],
              [sg.Button('Logout')],
              [sg.Button('Return To Main')]]
    window = sg.Window("Deleted!",layout)
    

    
def button_update_worker(values):
    global window
    global up_worker
    
    new_Salary = values['salary']
    new_Shift = values['shift_time']
    new_Team = values['team_no']
    ISN = up_worker[0][0]
    old_salary = up_worker[0][3]
    old_shift = up_worker[0][1]
    old_team = up_worker[0][2]

              
    if new_Salary == '':
        if new_Shift == '':
            if new_Team == '':
                sg.popup('At least one box should be filled to update.')
                window = sg.Window("Workers")
            else:
                cur.execute('UPDATE WORKER SET Salary = ? , Shift_time = ? , Team_no = ?  WHERE WISN = ?',(old_salary, old_shift, new_Team, ISN,))
                con.commit()
            
                
        elif new_Team == '':
            cur.execute('UPDATE WORKER SET Salary = ? , Shift_time = ? , Team_no = ?  WHERE WISN = ?',(old_salary, new_Shift, old_team, ISN,))
            con.commit()
        else:
            cur.execute('UPDATE WORKER SET Salary = ? , Shift_time = ? , Team_no = ?  WHERE WISN = ?',(old_salary, new_Shift, new_Team, ISN,))
            con.commit()
            
    elif not new_Salary.isnumeric():
        sg.popup('Salary should be numeric.')
    elif new_Shift == '':
        if new_Team == '':
            cur.execute('UPDATE WORKER SET Salary = ? , Shift_time = ? , Team_no = ? WHERE WISN = ?',(new_Salary, old_shift, old_team, ISN,))
            con.commit()
        else:
            cur.execute('UPDATE WORKER SET Salary = ? , Shift_time = ?, Team_no = ?  WHERE WISN = ?',(new_Salary, old_shift, new_Team, ISN,))
            con.commit()
    elif new_Team == '':
        cur.execute('UPDATE WORKER SET Salary = ? , Shift_time = ? , Team_no = ? WHERE WISN = ?',(new_Salary, new_Shift, old_team, ISN,))
        con.commit()
    
    else:
        cur.execute('UPDATE WORKER SET Salary = ? , Shift_time = ? , Team_no = ?  WHERE WISN = ?',(new_Salary, new_Shift, new_Team, ISN,))
        con.commit()
    
        sg.popup('Successfully updated '+ ISN)
    
    button_workers(values)
              
def window_update_worker(values):
    global window 
    global up_worker
    
    up_worker = values["workers"]
    if values['workers'] == [] or values['workers'] == [[]] or values['workers'] == "":
        sg.popup('Please Select a Worker')
        button_workers(values = None)
        return 

    
    layout = [[sg.Text('Salary',size=(10,1)) , sg.Input(key='salary',size=(10,1))],
              [sg.Text('Shift Time',size=(10,1)) , sg.Input(key='shift_time',size=(10,1))],
              [sg.Text('Team No',size=(10,1)) , sg.Input(key='team_no',size=(10,1))],
              [sg.Button('Update Worker Information') , sg.Button('Return To Main')]]
    
    
    window = sg.Window('Update Worker Information', layout)
    
    
def button_insert_new_project_director(values):
    global window 
    
    
    Name = values['Name']
    Surname = values['Surname']
    S_no = values['s_no']
    Work_hours = values['work_hours']
    Experience = values['experience']
    salary= values['salary']
    ISN = ""
    for i in range(8):
        ISN += str(random.randint(0,9))
    password = ""
    for i in range(5):
        password+=str(random.randint(0,9))
    if Name == '':
        sg.popup('Name cannot be empty!')
    elif Surname == '':
        sg.popup('Surname cannot be empty!')
    elif salary == '':
        sg.popup('Please write the salary amount. Write 0 for no salary.')
    elif not salary.isnumeric():
        sg.popup('Salary should be numeric.')
    elif not Work_hours.isnumeric():
        sg.popup('Work hours should be numeric.')
    else:
        
       
        try:
            cur.execute('INSERT INTO USER VALUES (?,?,?,?)',(ISN,Name,Surname,password,))
            con.commit()
            cur.execute('INSERT INTO PROJECT_DIRECTOR VALUES (?,?,?,?,?)',(ISN,salary,S_no,Work_hours,Experience,))
            con.commit()

            sg.popup('Successfully inserted ' + Name + ' ' + Surname + ' with id: ' + str(ISN) + ', password is ' + password)
        except: 
            sg.popup('ISN must be unique, try again')
        
    window_insert_project_director(values)
    
def window_insert_project_director(values):
    global window
    
    layout = [[sg.Text('Name:',size=(10,1)) , sg.Input(key='Name',size=(10,1))],
              [sg.Text('Surname:',size=(10,1)) , sg.Input(key='Surname',size=(10,1))],
              [sg.Text('S_no',size=(10,1)) , sg.Input(key='s_no',size=(10,1))],
              [sg.Text('Work_hours',size=(10,1)) , sg.Input(key='work_hours',size=(10,1))],
              [sg.Text('Experience',size=(10,1)) , sg.Input(key='experience',size=(10,1))],
              [sg.Text('Salary',size=(10,1)) , sg.Input(key='salary',size=(10,1))],
              [sg.Button('Insert New Project Director') , sg.Button('Return To Main')]]
    
    
    window = sg.Window('Insert Project Director', layout)
    
    
def button_select(values):
    global window 
    global team_no
    
    if values['projects'] == [] or values['projects'] == [[]] or values['projects'] == "":
        sg.popup('Please Select a Project')
        button_spacecraft_projects(values = None)
        return 
        
    team_no = values['projects'][0][4]
    
    window_rate(values)
   
    
def window_rate(values):
    global window 
    global team_no
    
    inspected_workers = []
    inspected_workers.append(('WISN','Shift_time', 'Team_no', 'Salary'))
    for row in cur.execute('SELECT WISN, Shift_time, Team_no, Salary FROM WORKER WHERE Team_no= ?',(team_no, )):
        inspected_workers.append(row)
        
        
    layout=   [[sg.Text('Inspected Workers')],
              [sg.Listbox(inspected_workers,size=(40,7),key='inspected_workers')],
              [sg.Text('Rate',size=(10,1)), sg.Input(size=(10,1), key='rate')],
              [sg.Button('Return To Main'),sg.Button('Rate')]]
    
    window = sg.Window('Inspected Workers',layout)
    
    
def button_rate(values):
    global window 
    global userIsn 
    global Rating
    
    GMISN = userIsn 
    WISN = values['inspected_workers'][0][0]
    Rating = values['rate']
    cursor = cur.execute('SELECT Salary FROM WORKER WHERE WISN = ?', (WISN,))
    Salary = cursor.fetchall()[0]
        
    if Rating == '':
        sg.popup('Please write the rating.')
    elif not Rating.isnumeric():
        sg.popup('Rating should be numeric.')
    else:
        cur.execute('INSERT INTO INSPECT VALUES(?,?,?)',(WISN, GMISN, Rating,  ))
        con.commit()
        if int(Rating) >= 5: 
            bonus_salary = int(Salary[0]+(int(Salary[0])*0.1))
            cur.execute('UPDATE WORKER SET Salary = ? WHERE WISN =?',(bonus_salary, WISN, ))
            con.commit()
    
        sg.popup('Successfully rated and Bonus has been added to the salary.')
    
    window_rate(values)

def button_view_ratings(values = None):
    global window
    global userIsn
    
    ratings = []
    ratings.append(('WISN', 'GMISN', 'Rating'))
    for row in cur.execute('SELECT WISN, GMISN, Rating FROM INSPECT WHERE WISN =?', (userIsn,)):
        ratings.append(row)
        
    layout = [[sg.Text('Ratings')],
              [sg.Listbox(ratings,size=(40,7),key='ratings')],
              [sg.Button('Return To Main')]]
    
    window = sg.Window('View Ratings',layout)
    

def button_list_spacecrafts(filter:bool, values = None):
    global window
    spaceCrafts = []
    if filter:
        if (values['brand'] == "" or values['brand'] == " " ) and (values['uprice'] == "" or values['uprice'] == " " ) and\
            (values['lprice'] == "" or values['lprice'] == " " )\
            and (values['useats'] == "" or values['useats'] == " " ) and (values['lseats'] == "" or values['lseats'] == " " ):
            sg.popup('Please enter valid preferences before filtering')
            window.close()
            button_list_spacecrafts(False)
            return 
        else:
            up = values['uprice']
            lp = values['lprice']
            ls = values['lseats']
            us = values['useats']
            if up == "":
                up = str(sys.maxsize)
            if lp == "":
                lp = str(0)
            if us == "":
                us = str(1000)
            if ls  == "":
                ls = str(0)
                
            if values['brand'] == "":
                spaceCrafts.append(('PLATE', 'SEATS', 'BRAND', 'PRICE'))
                for row in cur.execute('''SELECT Lic_Plate, Num_Seats, Brand, Price
                FROM SPACE_CRAFT_CREATION WHERE Price <= ? and Price >= ? and Num_Seats <= ? 
                and Num_Seats >= ?''',(up,lp,us,ls,)):
                    spaceCrafts.append(row)
            else:    
                
                spaceCrafts.append(('PLATE', 'SEATS', 'BRAND', 'PRICE'))
                for row in cur.execute('''SELECT Lic_Plate, Num_Seats, Brand, Price
                                          FROM SPACE_CRAFT_CREATION WHERE upper(Brand) = ? 
                                          and Price <= ? and Price >= ? and Num_Seats <= ? 
                                          and Num_Seats >= ?''',(values['brand'].upper(),up,lp,us,ls,)):
                    spaceCrafts.append(row)

            
    else:    
        spaceCrafts.append(('PLATE', 'SEATS', 'BRAND', 'PRICE'))
        for row in cur.execute('SELECT Lic_Plate, Num_Seats, Brand, Price FROM SPACE_CRAFT_CREATION'):
            spaceCrafts.append(row)
        
    layout = [[sg.Text('Spacecrafts')],
              [sg.Listbox(spaceCrafts,size=(40,7),key='spacecrafts')],
              [sg.Text('Brand:',size=(10,1)), sg.Input(size=(10,1), key='brand')],
              [sg.Text('Min Price:',size=(10,1)), sg.Input(size=(10,1), key='lprice'),sg.Text('Max Price:',size=(10,1)),sg.Input(size=(10,1), key='uprice')],
              [sg.Text('Min Seats:',size=(10,1)), sg.Input(size=(10,1), key='lseats'),sg.Text('Max Seats:',size=(10,1)),sg.Input(size=(10,1), key='useats')],
              [sg.Button('Filter Spacecrafts'),sg.Button('Purchase Selected')],
              [sg.Button('Return To Main')]]
    
    window = sg.Window('Spacecrafts',layout)
    

def button_purchase(values):
    global purchased
    global window
    global temp_purchase
    global inited
    
    if len(values['spacecrafts']) == 0:
        sg.popup('Please select a spacecraft to buy')
        window.close()
        button_list_spacecrafts(False)
        return 
    else:
        invoice = ""
        for i in range(6):
            invoice += str(random.randint(0,9))
        temp_purchase = values['spacecrafts']
        l = [x for x in temp_purchase[0]]
        l.append(invoice)
        temp_purchase = tuple(l)
        representatives = []
        representatives.append(('NAME','SURNAME','RATE'))
        for row in cur.execute('SELECT NAME, SURNAME,Com_rate,SRISN FROM USER,SALES_REPRESENTATIVE WHERE USER.ISN = SALES_REPRESENTATIVE.SRISN'):
            representatives.append(row)
        if not inited:
            for x in representatives:
                purchased[x[-1]] = {}
            inited = True
        
        layout = [[sg.Text('Please select a sales representative who will finalize your purchase process.')],
              [sg.Text('Available Sales Representatives:')],
              [sg.Combo(representatives,size=(40,7),key='available_representatives')],
              [sg.Button('Select Representative')],  
              [sg.Button('Return To Main')]]
        window = sg.Window('Sales Representative Selection',layout)
        
def button_select_representative(values):
    global window
    global temp_purchase
    global purchased
    
    if len(values['available_representatives']) == 0:
        sg.popup('Please select a Sales Representative!')
    else:
        
        purchased[values['available_representatives'][-1]][temp_purchase[-1]] = temp_purchase
        #temp_purchase = None
        sg.popup('Congratulations!')
        layout = [[sg.Text("Your Purchase Process Will be Finalized by your Sales Representative")],
                  [sg.Button('Logout')]]
        window = sg.Window("Congratulations!", layout)
        
        
def button_list_purchases(filter:bool, values = None):
    global window
    if filter:
        
        if values['searchisn'] == "":
            sg.popup('Please enter valid preferences before filtering')
            operations = []
            operations.append(('Invoice', 'Plate', 'SR_ISN', 'Tr_date'))
            for row in cur.execute('SELECT Invoice, Plate, SR_ISN, Tr_date FROM PURCHASE'):
                operations.append(row)
            layout = [[sg.Text('Purchase Opreations')],
              [sg.Listbox(operations,size=(40,7),key='operations')],
              [sg.Text('SRISN:',size=(10,1)), sg.Input(size=(10,1), key='searchisn')],
              [sg.Button('Filter Operations')],
              [sg.Button('Return To Main')]]
            window = sg.Window('Purchase Operations',layout)
            return
        else:
            operations = []
            operations.append(('Invoice', 'Plate', 'SR_ISN', 'Tr_date'))
            for row in cur.execute('''SELECT Invoice, Plate, SR_ISN, Tr_date
                                      FROM PURCHASE WHERE SR_ISN = ?''',(values['searchisn'],)):
                operations.append(row)
            
            
    else:    
        operations = []
        operations.append(('Invoice', 'Plate', 'SR_ISN', 'Tr_date'))
        for row in cur.execute('SELECT Invoice, Plate, SR_ISN, Tr_date FROM PURCHASE'):
            operations.append(row)
        
    layout = [[sg.Text('Purchase Opreations')],
              [sg.Listbox(operations,size=(40,7),key='operations')],
              [sg.Text('SRISN:',size=(10,1)), sg.Input(size=(10,1), key='searchisn')],
              [sg.Button('Filter Operations')],
              [sg.Button('Return To Main')]]
    
    window = sg.Window('Purchase Operations',layout)
            
def button_pending():
    global window
    global purchased
    global inited
    representatives =  []
    for row in cur.execute('SELECT NAME, SURNAME,Com_rate,SRISN FROM USER,SALES_REPRESENTATIVE WHERE USER.ISN = SALES_REPRESENTATIVE.SRISN'):
        representatives.append(row)
    if not inited:
        for x in representatives:
            purchased[x[-1]] = {}
        inited = True
    
    
    if len(purchased[userIsn].keys()) == 0:
        sg.popup('No Pending Operations For You')
        window = window_salesRep()
    else:
        operations = [purchased[userIsn][x] for x in purchased[userIsn].keys()]
        layout = [[sg.Text('Your Purchase Operations')],
                  [sg.Listbox(operations,size=(40,7),key='operations')],
                  [sg.Button('Confirm'),sg.Button('Reject')],
                  [sg.Button('Return To Main')]]

        window = sg.Window('Pending Operations',layout) 

def button_confirm_purchase(values):
    global window
    global purchased
    
    
    try:
        cur.execute('INSERT INTO PURCHASE VALUES (?,?,?,?)',(values['operations'][0][-1],values['operations'][0][0],userIsn, 
               '2150-12-22'),)
        con.commit()
    except sqlite3.IntegrityError as e:
        print('sqlite error: ', e.args[0])
    del purchased[userIsn][values['operations'][0][-1]]
        
    sg.popup('Congratulations')
    layout = [[sg.Text('Operation is Confirmed')],
              [sg.Button('Logout')],
              [sg.Button('Return To Main')]]
    window = sg.Window("Confirmed!",layout)

def button_reject_purchase(values):
    global window
    global pruchased

    del purchased[userIsn][values['operations'][0][-1]]
        
    sg.popup('Succesfuly Rejected')
    layout = [[sg.Text('Operation is Rejected')],
              [sg.Button('Logout')],
              [sg.Button('Return To Main')]]
    window = sg.Window("Rejected!",layout)
    
def button_listprojects():
    global window
    Projects=[]            
    Projects.append(( 'Pr_No', 'Team_No'))
    for row in cur.execute('''SELECT Pr_No, Team_No FROM TEAM, PROJECT_DIRECTOR WHERE PROJECT_DIRECTOR.PDISN = ? and TEAM.PD_ISN = PROJECT_DIRECTOR.PDISN  ''', (userIsn,)):
        Projects.append(row)
    
    layout = [[sg.Text(userName + " "+ userSurname + " " + userIsn)],
              [sg.Listbox(Projects,size=(40,7),key='projects')],
              [sg.Button('Create a Project'), sg.Button('Return To Main')],
              [sg.Button('Logout')]]
    

    window = sg.Window('Project Director Window', layout)
def button_createproject():
        global window
        layout = [[sg.Text('Select a Project Number:'), sg.Input(size=(10,1), key = 'pr_no')],
                  [sg.Button('Create a Team')],  
                  [sg.Button('Return To Main')]]
        window = sg.Window('Project Numberization',layout)

def button_createteam(assigned:bool, values):
    global Project_Number
    global Team_Number
    global assigned_workers
    global window
    available_workers = []
    available_workers.append(('WISN', 'Shift Time', 'Salary'))
    
    if assigned:
        for row in cur.execute('''SELECT WISN, Shift_Time, Salary
        FROM WORKER WHERE Team_No IS NULL'''):
            if row not in assigned_workers:
                available_workers.append(row)
        layout = [[sg.Text('Available Workers:')],
                  [sg.Listbox(available_workers, size=(40,7), key = 'worker')],
                  [sg.Button('Assign This Worker To This Team'), sg.Button('Finish the team')],
                  [sg.Button('Return To Main')]]
        window = sg.Window('Project Creation',layout)
    
    else:
        Team_Number = None
        assigned_workers = []
        for row in cur.execute('''SELECT WISN, Shift_Time, Salary
        FROM WORKER WHERE Team_No IS NULL'''):
            available_workers.append(row)
        if Project_Number == None:
            pr_no = values['pr_no']
            if pr_no == '':
                sg.popup('Project Number Can not be Empty!')
                window.close()
                button_createproject()
            elif int(pr_no) > 100:
                sg.popup('Project Number can be between 6 and 100')
                window.close()
                button_createproject()
            else: 
                cur.execute('SELECT Pr_No FROM TEAM WHERE Pr_No = ?', (pr_no,))
                row_projectno = cur.fetchone()
                if row_projectno == None:
                    Project_Number = pr_no

                    layout = [[sg.Text('Available Workers:')],
                              [sg.Listbox(available_workers, size=(40,7), key = 'worker')],
                              [sg.Text('Indicate a team number:'), sg.Input(size=(10,1), key = 'team_no')],
                              [sg.Button('Assign This Worker To This Team'), sg.Button('Finish the team')],
                              [sg.Button('Return To Main')]]  
                    window = sg.Window('Project Creation', layout)
                else:
                    sg.popup('This project number has already been taken. Select another one.')
                    window.close()
                    button_createproject()
        else:
            layout = [[sg.Text('Available Workers:')],
                      [sg.Listbox(available_workers, size=(40,7), key = 'worker')],
                      [sg.Text('Indicate a team number:'), sg.Input(size=(10,1), key = 'team_no')],
                      [sg.Button('Assign This Worker To This Team'), sg.Button('Finish the team')],
                      [sg.Button('Return To Main')]]  
            window = sg.Window('Project Creation', layout)
def button_existingassignment(values):
    global window
    
    available_workers = []
    available_workers.append(('WISN', 'Shift Time', 'Salary'))
    for row in cur.execute('''SELECT WISN, Shift_Time, Salary
        FROM WORKER WHERE Team_No IS NULL'''):
        available_workers.append(row)
    layout = [[sg.Text('Available Workers:')],
              [sg.Listbox(available_workers, size=(40,7), key = 'worker')],
              [sg.Text('Assign to team (insert an existing team number): '), sg.Input(size=(10,1), key = 'team_no')],
              [sg.Button('Assign'), sg.Button('Return To Main')]]
    window = sg.Window('Existing Assignment', layout)
def button_assign(values):
    tno = values['team_no']
    
    if tno == '':
        sg.popup('Team Number Can not be Empty!')
        window.close()
        button_existingassignment(values)
    elif len(values['worker']) == 0:
        sg.popup('Please Select a Worker!')
        window.close()
        button_existingassignment(values)
        
    else:
        cur.execute('SELECT Team_No FROM TEAM WHERE Team_No = ?', (tno,))
        row = cur.fetchone()
        if row is None:
            sg.popup('This Team Does Not Exist! Please Insert an Existing Team.')
            window.close()
            button_existingassignment(values)
        else:
            cur.execute('UPDATE WORKER SET Team_No = ? WHERE WISN = ?', (tno, values['worker'][0][0],))
            con.commit()
            sg.popup('You have successfully assigned this worker to Team ' + tno)
            window.close()
            button_existingassignment(values)
             
        

              
    

def button_assignworker(values):
    global Project_Number
    global Team_Number
    global assigned_workers
    global window
    
    if Team_Number == None:
        tno = values['team_no']
        if tno == '':
            sg.popup('Team Number Can Not be Empty')
            window.close()
            button_createteam(False, values)
        elif len(values['worker']) == 0:
            sg.popup('Please Select a Worker')
            window.close()
            button_createteam(False, values)
        else:
            cur.execute('SELECT Team_No FROM TEAM WHERE Team_No = ?', (tno,))
            row = cur.fetchone()
            if row is None:
                Team_Number = tno
                assigned_workers.append(values['worker'][0]) 

                sg.popup('You Have Successfully Selected a Member for Team ' + Team_Number + ' and Project ' + Project_Number)
                window.close()
                button_createteam(True,values)

            else:
                sg.popup('This team number already exists, select a unique one')
                button_createteam(False, values)
    else:
        if len(values['worker']) == 0:
            sg.popup('Please Select a Worker')
            window.close()
            button_createteam(True, values)
        else:
            assigned_workers.append(values['worker'][0])
            sg.popup('You Have Successfully Selected a Member for Team ' + Team_Number + ' and Project ' + Project_Number)
            window.close()
            button_createteam(True, values)
def button_finalizeteam(values):
    global assigned_workers
    global Team_Number
    global userIsn
    global Project_Number
    global window
    if assigned_workers == []:
        sg.popup('You have not assigned any workers yet!')
        window.close()
        button_createteam(False, values)
    else:    
        for worker in assigned_workers:
            cur.execute('UPDATE WORKER SET Team_No = ? WHERE WISN = ?', (Team_Number, worker[0],))
            update = cur.fetchone()
            con.commit()
    
        cur.execute('INSERT INTO TEAM VALUES (?,?,?)', (Team_Number, userIsn, Project_Number,))
        con.commit()
        sg.popup('Congratulations! You have created Project: ' + Project_Number + ' Team: ' + Team_Number)
        window.close()
        window = window_projectDirector()
                
    assigned_workers = []
            
    

    
    
    


In [5]:
userIsn = None
userType = None
userName = None
userSurname = None
window = window_login()

purchased = {}
temp_purchase = None
inited = False
up_worker = None
team_no = None
Rating = None
Project_Number = None
Team_Number = None
assigned_workers = []


while True:
    event, values = window.read()
    if event == 'Login':
        window.close()
        button_login(values)
    elif event == 'Purchase Operations':
        window.close()
        button_list_purchases(values)
    elif event == 'Project Directors':
        window.close()
        button_project_directors(values)
    elif event == 'Spacecraft Projects':
        window.close()
        button_spacecraft_projects(values)
    elif event == 'Workers':
        window.close()
        button_workers(values)
    elif event == 'Manufactured Spacecrafts':
        window.close()
        button_spacecrafts(values)
    elif event == 'Insert New Worker':
        window.close()
        window_insert_worker(values)
    elif event == 'Insert Worker':
        window.close()
        button_insert_worker(values)
    elif event == 'Delete Worker':
        window.close()
        button_delete_worker(values)
    elif event == 'Update Worker':
        window.close()
        window_update_worker(values)
    elif event == 'Update Worker Information':
        window.close()
        button_update_worker(values)
    elif event == 'Insert Project Director':
        window.close()
        window_insert_project_director(values)
    elif event == 'Insert New Project Director':
        window.close()
        button_insert_new_project_director(values)
    elif event == 'Select':
        window.close()
        button_select(values)
    elif event == 'Rate':
        window.close()
        button_rate(values)
    elif event == 'View Ratings':
        window.close()
        button_view_ratings(values)
    elif event == 'Spacecrafts':
        window.close()
        button_list_spacecrafts(False)
    elif event == 'Filter Spacecrafts':
        window.close()
        button_list_spacecrafts(True,values)
    elif event == 'Purchase Selected':
        window.close()
        button_purchase(values)
    elif event == 'Select Representative': 
        window.close()
        button_select_representative(values)
    elif event == 'Purchase Operations1':
        window.close()
        button_purchases(False)
    elif event == 'Filter Operations':
        window.close()
        button_list_purchases(True, values)
    elif event == 'Pending Purchase Operations':
        window.close()
        button_pending()
    elif event == 'Confirm':
        window.close()
        button_confirm_purchase(values)
    elif event == 'Reject':
        window.close()
        button_reject_purchase(values)
    elif event == 'List My Projects':
        window.close()
        button_listprojects()
    elif event == 'Create a Project':
        window.close()
        button_createproject()
    elif event == 'Create a Team':
        window.close()
        button_createteam(False, values)
    elif event == 'Assign This Worker To This Team':
        window.close()
        button_assignworker(values)
    elif event == 'Finish the team':
        window.close()
        button_finalizeteam(values)
    elif event == 'Assign Workers to Teams':
        window.close()
        button_existingassignment(values)
    elif event == 'Assign':
        window.close()
        button_assign(values)
    elif event == 'Return To Main':
        if userType == 'Manager':
            window.close()
            window = window_manager()
        elif userType == 'Worker':
            window.close()
            window = window_worker()
        elif userType == 'Customer':
            window.close()
            window = window_customer()
        elif userType == 'Sales Representative':
            window.close()
            window = window_salesRep()
        elif userType == 'Project Director':
            window.close()
            window = window_projectDirector()
        
        else:
           
            window.close()
            window = window_login()    
        
    elif event == 'Logout':       
        userIsn = None
        userType = None
        userName = None
        userSurname = None
        window.close()
        window = window_login()
    elif event == sg.WIN_CLOSED:
        break